# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 17 2022 10:55AM,246671,22,627613,"NBTY Global, Inc.",Released
1,Sep 17 2022 9:39AM,246670,20,8537531-BO,"ACI Healthcare USA, Inc.",Released
2,Sep 17 2022 9:39AM,246670,20,8537532-BO,"ACI Healthcare USA, Inc.",Released
3,Sep 17 2022 9:39AM,246670,20,8537533-BO,"ACI Healthcare USA, Inc.",Released
4,Sep 17 2022 9:39AM,246670,20,8537534-BO,"ACI Healthcare USA, Inc.",Released
5,Sep 17 2022 9:39AM,246670,20,8537535-BO,"ACI Healthcare USA, Inc.",Released
6,Sep 17 2022 9:39AM,246670,20,8537536-BO,"ACI Healthcare USA, Inc.",Released
7,Sep 17 2022 9:39AM,246670,20,8537537-BO,"ACI Healthcare USA, Inc.",Released
8,Sep 17 2022 9:39AM,246670,20,8537538-BO,"ACI Healthcare USA, Inc.",Released
9,Sep 17 2022 9:39AM,246670,20,8537539-BO,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
6,246660,Released,1
7,246663,Released,1
8,246669,Released,4
9,246670,Released,19
10,246671,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Released
id,,
246660,NaN,1.0
246663,NaN,1.0
246669,NaN,4.0
246670,NaN,19.0
246671,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Released
id,,
246647,0.0,1.0
246648,0.0,1.0
246650,14.0,1.0
246657,0.0,4.0
246659,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,Completed,Released
id,,
246647,0,1
246648,0,1
246650,14,1
246657,0,4
246659,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Released
0,246647,0,1
1,246648,0,1
2,246650,14,1
3,246657,0,4
4,246659,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,id,Completed,Released
0,246647,,1
1,246648,,1
2,246650,14,1
3,246657,,4
4,246659,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 17 2022 10:55AM,246671,22,"NBTY Global, Inc."
1,Sep 17 2022 9:39AM,246670,20,"ACI Healthcare USA, Inc."
20,Sep 17 2022 8:50AM,246669,20,"ACI Healthcare USA, Inc."
24,Sep 16 2022 4:35PM,246647,21,"NBTY Global, Inc."
25,Sep 16 2022 4:35PM,246648,21,"NBTY Global, Inc."
26,Sep 16 2022 4:35PM,246659,21,"NBTY Global, Inc."
27,Sep 16 2022 4:35PM,246660,21,"NBTY Global, Inc."
28,Sep 16 2022 4:34PM,246663,21,"NBTY Global, Inc."
29,Sep 16 2022 3:48PM,246657,20,Alumier Labs Inc.
33,Sep 16 2022 3:17PM,246650,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Released
0,Sep 17 2022 10:55AM,246671,22,"NBTY Global, Inc.",,1
1,Sep 17 2022 9:39AM,246670,20,"ACI Healthcare USA, Inc.",,19
2,Sep 17 2022 8:50AM,246669,20,"ACI Healthcare USA, Inc.",,4
3,Sep 16 2022 4:35PM,246647,21,"NBTY Global, Inc.",,1
4,Sep 16 2022 4:35PM,246648,21,"NBTY Global, Inc.",,1
5,Sep 16 2022 4:35PM,246659,21,"NBTY Global, Inc.",,1
6,Sep 16 2022 4:35PM,246660,21,"NBTY Global, Inc.",,1
7,Sep 16 2022 4:34PM,246663,21,"NBTY Global, Inc.",,1
8,Sep 16 2022 3:48PM,246657,20,Alumier Labs Inc.,,4
9,Sep 16 2022 3:17PM,246650,15,"Person & Covey, Inc.",14,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Completed
0,Sep 17 2022 10:55AM,246671,22,"NBTY Global, Inc.",1,
1,Sep 17 2022 9:39AM,246670,20,"ACI Healthcare USA, Inc.",19,
2,Sep 17 2022 8:50AM,246669,20,"ACI Healthcare USA, Inc.",4,
3,Sep 16 2022 4:35PM,246647,21,"NBTY Global, Inc.",1,
4,Sep 16 2022 4:35PM,246648,21,"NBTY Global, Inc.",1,
5,Sep 16 2022 4:35PM,246659,21,"NBTY Global, Inc.",1,
6,Sep 16 2022 4:35PM,246660,21,"NBTY Global, Inc.",1,
7,Sep 16 2022 4:34PM,246663,21,"NBTY Global, Inc.",1,
8,Sep 16 2022 3:48PM,246657,20,Alumier Labs Inc.,4,
9,Sep 16 2022 3:17PM,246650,15,"Person & Covey, Inc.",1,14


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Sep 17 2022 10:55AM,246671,22,"NBTY Global, Inc.",1,
1,Sep 17 2022 9:39AM,246670,20,"ACI Healthcare USA, Inc.",19,
2,Sep 17 2022 8:50AM,246669,20,"ACI Healthcare USA, Inc.",4,
3,Sep 16 2022 4:35PM,246647,21,"NBTY Global, Inc.",1,
4,Sep 16 2022 4:35PM,246648,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet


,Date,id,Warehouse,Customer,Released,Completed
0,Sep 17 2022 10:55AM,246671,22,"NBTY Global, Inc.",1,NaN
1,Sep 17 2022 9:39AM,246670,20,"ACI Healthcare USA, Inc.",19,NaN
2,Sep 17 2022 8:50AM,246669,20,"ACI Healthcare USA, Inc.",4,NaN
3,Sep 16 2022 4:35PM,246647,21,"NBTY Global, Inc.",1,NaN
4,Sep 16 2022 4:35PM,246648,21,"NBTY Global, Inc.",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Sep 17 2022 10:55AM,246671,22,"NBTY Global, Inc.",1,0.0
1,Sep 17 2022 9:39AM,246670,20,"ACI Healthcare USA, Inc.",19,0.0
2,Sep 17 2022 8:50AM,246669,20,"ACI Healthcare USA, Inc.",4,0.0
3,Sep 16 2022 4:35PM,246647,21,"NBTY Global, Inc.",1,0.0
4,Sep 16 2022 4:35PM,246648,21,"NBTY Global, Inc.",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Completed
Warehouse,,,
15,246650,1,14.0
20,739996,27,0.0
21,1233277,5,0.0
22,246671,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Completed
0,15,246650,1,14.0
1,20,739996,27,0.0
2,21,1233277,5,0.0
3,22,246671,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Completed
0,15,1,14.0
1,20,27,0.0
2,21,5,0.0
3,22,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,15,Released,1.0
1,20,Released,27.0
2,21,Released,5.0
3,22,Released,1.0
4,15,Completed,14.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,15,20,21,22
Status,,,,
Completed,14.0,0.0,0.0,0.0
Released,1.0,27.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,15,20,21,22
0,Completed,14.0,0.0,0.0,0.0
1,Released,1.0,27.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,15,20,21,22
0,Completed,14.0,0.0,0.0,0.0
1,Released,1.0,27.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()